In [12]:
import torch
from torch import nn
from torch.nn import functional as F

In [9]:
net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X=torch.rand(2,20)
net(X)

tensor([[-0.2863,  0.0314,  0.0367,  0.0295,  0.0282, -0.0873,  0.3054,  0.0278,
         -0.2123,  0.1696],
        [-0.3334, -0.1171, -0.0461,  0.1564,  0.0780, -0.1487,  0.2850,  0.0367,
         -0.1791,  0.0771]], grad_fn=<AddmmBackward>)

在这个例子中，我们通过实例化nn.Sequential来构建我们的模型，层的执行顺序是作为参数传递的。简而言之，nn.Sequential定义了一种特殊的Module，即在PyTorch中表示一个块的类。它维护了一个由Module组成的有序列表，注意，两个全连接层都是Linear类的实例，Linear类本身就是Module的子类。正向传播（forward）函数也非常简单：它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。注意，到目前为止，我们一直在通过net(X)调用我们的模型来获得模型的输出。这实际上是net.__call__(X)的简写。

自定义块

要想直观地了解块是如何工作的，最简单的方法可能就是自己实现一个。在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能：

1 将输入数据作为其正向传播函数的参数。

2 通过正向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收任意维的输入，但是返回一个维度256的输出。

3 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

4 存储和访问正向传播计算所需的参数。

5 根据需要初始化模型参数。

在下面的代码片段中，我们从零开始编写一个块。它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。注意，下面的MLP类继承了表示块的类。我们的实现将严重依赖父类，只需要提供我们自己的构造函数（Python中的__init__函数）和正向传播函数。

In [26]:
class MLP(nn.Module):
    #用模型参数声明层，这里我们声明两个全连接层
    def __init__(self):
        super().__init__()
        self.hidden =nn.Linear(20,256)
        self.out=nn.Linear(256,10)

    #定义模型的正向传播，即如何根输入X返回所需要的模型输出
    def forward(self,X):
        #注意，这里使用relu的函数版本，在nn.functional中定义
        return self.out(F.relu(self.hidden(X)))

让我们首先关注正向传播函数。注意，它以X作为输入，计算带有激活函数的隐藏表示，并输出其未归一化的输出值。在这个MLP实现中，两个层都是实例变量。要了解这为什么是合理的，可以想象实例化两个多层感知机（net1和net2），并根据不同的数据对它们进行训练。当然，我们希望它们学到两种不同的模型。

我们在构造函数中实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层。注意一些关键细节。首先，我们定制的__init__函数通过super().__init__()调用父类的__init__函数，省去了重复编写适用于大多数块的模版代码的痛苦。然后我们实例化两个全连接层，分别为self.hidden和self.out。注意，除非我们实现一个新的运算符，否则我们不必担心反向传播函数或参数初始化，系统将自动生成这些。让我们试一下。

In [27]:
net =MLP()
net(X)

tensor([[-0.1050,  0.3085, -0.0962, -0.0542, -0.0556,  0.2408, -0.2055, -0.2086,
          0.0416, -0.0475],
        [ 0.1397,  0.3354, -0.0488, -0.0443,  0.0360,  0.1239, -0.2923, -0.2438,
          0.0111,  0.0961]], grad_fn=<AddmmBackward>)

顺序块

现在我们可以更仔细地看看Sequential类是如何工作的。回想一下Sequential的设计是为了把其他模块串起来。为了构建我们自己的简化的MySequential，我们只需要定义两个关键函数： 1. 一种将块逐个追加到列表中的函数。 2. 一种正向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的MySequential类提供了与默认Sequential类相同的功能。

In [14]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] =block
    
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X

在__init__方法中，我们将每个块逐个添加到有序字典_modules中。你可能会想知道为什么每个Module都有一个_modules属性，以及为什么我们使用它而不是自己定义一个Python列表。简而言之，_modules的主要优点是，在块的参数初始化过程中，系统知道在_modules字典中查找需要初始化参数的子块。

当MySequential的正向传播函数被调用时，每个添加的块都按照它们被添加的顺序执行。现在可以使用我们的MySequential类重新实现多层感知机。

In [16]:
net =MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.1140,  0.1305,  0.0548, -0.0300,  0.0697,  0.1774,  0.2238, -0.1214,
          0.0321, -0.1764],
        [ 0.1703,  0.0009,  0.1570, -0.0126, -0.0349,  0.1388,  0.2823, -0.2771,
         -0.1137, -0.2407]], grad_fn=<AddmmBackward>)

在正向传播中执行代码

Sequential类使模型构造变得简单，允许我们组合新的结构，而不必定义自己的类。然而，并不是所有的架构都是简单的顺序结构。当需要更大的灵活性时，我们需要定义自己的块。例如，我们可能希望在正向传播函数中执行Python的控制流。此外，我们可能希望执行任意的数学运算，而不是简单地依赖预定义的神经网络层。

你可能已经注意到，到目前为止，我们网络中的所有操作都对网络的激活值及网络的参数起作用。然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项。我们称之为常数参数（constant parameters）。例如，我们需要一个计算函数 f(x,w)=c⋅w⊤x 的层，其中 x 是输入， w 是我们的参数， c 是某个在优化过程中没有更新的指定常量。因此我们实现了一个FixedHiddenMLP类，如下所示。

In [22]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight =torch.rand(20,20,requires_grad=False)
        self.linear =nn.Linear(20,20)
    
    def forward(self,X):
        X=self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X=self.linear(X)

        #控制流 没有任何意义 就是告诉我们这里可以自己加入任意的自己想要加入的东西
        while X.abs().sum() >1:
            X/=2
        
        return X.sum()


在这个FixedHiddenMLP模型中，我们实现了一个隐藏层，其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。这个权重不是一个模型参数，因此它永远不会被反向传播更新。然后，网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，我们的模型做了一些不寻常的事情。我们运行了一个while循环，在 L1 范数大于 1 的条件下，将输出向量除以 2 ，直到它满足条件为止。最后，我们返回了X中所有项的和。据我们所知，没有标准的神经网络执行这种操作。注意，此特定操作在任何实际任务中可能都没有用处。我们的重点只是向你展示如何将任意代码集成到神经网络计算的流程中。

In [23]:
net =FixedHiddenMLP()
net(X)

tensor(0.2872, grad_fn=<SumBackward0>)